# Silver Layer - Data Cleaning with MapReduce

## Lending Club Loan Data Pipeline

**Use Case:** Predict loan default risk and analyze factors affecting loan approval

This notebook implements the Silver (Cleaned) layer of the Medallion Architecture:
- Clean and transform data using **RDD MapReduce operations** (no DataFrames/SQL)
- Handle missing values, type conversions, and data standardization
- Profile and tune performance

**Important:** As per project requirements, this notebook uses basic MapReduce routines in Spark (map, filter, reduce, reduceByKey, etc.) - NOT DataFrames or SQL.

## 1. Setup and Configuration

In [34]:
import time
import json
import re
import os
from datetime import datetime
from collections import defaultdict
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import *

def clean_percentage(value):
    if value is None or value == '': return None
    try: return float(str(value).replace('%', '').strip())
    except: return None

def clean_term(value):
    if value is None or value == '': return None
    try:
        match = re.search(r'(\d+)', str(value))
        return int(match.group(1)) if match else None
    except: return None

def clean_emp_length(value):
    if value is None or value == '' or str(value).lower() == 'n/a': return None
    val = str(value).lower()
    if '10+' in val: return 10
    if '< 1' in val: return 0
    try:
        match = re.search(r'(\d+)', val)
        return int(match.group(1)) if match else None
    except: return None

def clean_numeric(value):
    if value is None or str(value).lower() in ['null', 'nan', 'none', '']: return None
    try: return float(value)
    except: return None

def clean_string(value):
    if value is None: return None
    cleaned = str(value).strip()
    if not cleaned or cleaned.lower() == 'null': return None
    return cleaned

def clean_date(value):
    if value is None or value == '': return None
    try: return datetime.strptime(str(value), '%b-%Y').strftime('%Y-%m-%d')
    except: return None

def create_loan_status_binary(status):
    if status is None: return None
    s = str(status).lower().strip()
    if s in ['charged off', 'default', 'late (31-120 days)', 'does not meet the credit policy. status:charged off']: return 1
    if s in ['fully paid', 'current', 'in grace period']: return 0
    return None

def clean_row(row):
    d = row.asDict()
    return {
        'loan_amnt': clean_numeric(d.get('loan_amnt')),
        'term': clean_term(d.get('term')),
        'int_rate': clean_percentage(d.get('int_rate')),
        'grade': clean_string(d.get('grade')),
        'emp_length': clean_emp_length(d.get('emp_length')),
        'annual_inc': clean_numeric(d.get('annual_inc')),
        'loan_status_binary': create_loan_status_binary(d.get('loan_status'))
    }

print("Functions defined. Ready for benchmarking.")

Functions defined. Ready for benchmarking.


In [35]:
# Define the configurations to test
configs = {
    "Config 1 (Standard 4x4)": {
        "spark.executor.memory": "4g",
        "spark.executor.cores": "4",
        "spark.task.cpus": "1",
        "spark.default.parallelism": "16" 
    },
    "Config 2 (Fat Tasks)": {
        "spark.executor.memory": "4g",
        "spark.executor.cores": "4",
        "spark.task.cpus": "4", # One task takes whole executor (no parallelism)
        "spark.default.parallelism": "4"
    },
    "Config 3 (Tiny Executors)": {
        "spark.executor.memory": "1g",
        "spark.executor.cores": "1",
        "spark.task.cpus": "1",
        "spark.default.parallelism": "24" # High parallelism
    },
    "Config 4 (Balanced 2x2)": {
        "spark.executor.memory": "2g",
        "spark.executor.cores": "2",
        "spark.task.cpus": "1",
        "spark.default.parallelism": "16"
    }
}

# Define paths
BRONZE_PATH = "../data/medallion/bronze/accepted_loans"
results = {}

print("=== STARTING CONFIGURATION BENCHMARK ===")

for name, conf in configs.items():
    print(f"\nTesting: {name}")
    print(f"Settings: {conf}")
    
    # 1. Stop existing Spark session
    if 'spark' in globals():
        spark.stop()
        time.sleep(5) # Give cluster time to release resources
    
    # 2. Start new Spark session with specific config
    builder = SparkSession.builder \
        .appName(f"Silver-Benchmark-{name}") \
        .master("spark://spark-master:7077") \
        .config("spark.driver.memory", "4g") \
        .config("spark.sql.adaptive.enabled", "true")
    
    for k, v in conf.items():
        builder.config(k, v)
        
    spark = builder.getOrCreate()
    
    # 3. Run Workload (Read -> Map -> Filter -> Count)
    # We include a Shuffle (reduceByKey) to stress test the network config
    try:
        start_time = time.time()
        
        # Load
        df = spark.read.parquet(BRONZE_PATH)
        rdd = df.rdd
        
        # Heavy Processing
        res = rdd.map(clean_row) \
                 .filter(lambda x: x['loan_amnt'] is not None and x['loan_amnt'] > 1000) \
                 .map(lambda x: (x['grade'], 1)) \
                 .reduceByKey(lambda a, b: a + b) \
                 .collect()
                 
        duration = time.time() - start_time
        results[name] = duration
        print(f"Result: {duration:.2f} seconds")
        
    except Exception as e:
        print(f"Failed: {e}")
        results[name] = 9999.0

# 4. Print Final Leaderboard
print("\n" + "="*40)
print("BENCHMARK LEADERBOARD")
print("="*40)
sorted_results = sorted(results.items(), key=lambda x: x[1])

for rank, (name, time_taken) in enumerate(sorted_results, 1):
    print(f"{rank}. {name:<25} : {time_taken:.2f}s")

# 5. Initialize the winner for the rest of the notebook
winner_name = sorted_results[0][0]
winner_conf = configs[winner_name]
print(f"\nInitializing session with winner: {winner_name}")
spark.stop()
time.sleep(2)
builder = SparkSession.builder \
    .appName("LendingClub-Silver-Final") \
    .master("spark://spark-master:7077") \
    .config("spark.driver.memory", "4g")

for k, v in winner_conf.items():
    builder.config(k, v)

spark = builder.getOrCreate()
sc = spark.sparkContext
print("Ready to proceed with notebook!")

=== STARTING CONFIGURATION BENCHMARK ===

Testing: Config 1 (Standard 4x4)
Settings: {'spark.executor.memory': '4g', 'spark.executor.cores': '4', 'spark.task.cpus': '1', 'spark.default.parallelism': '16'}


Result: 25.13 seconds

Testing: Config 2 (Fat Tasks)
Settings: {'spark.executor.memory': '4g', 'spark.executor.cores': '4', 'spark.task.cpus': '4', 'spark.default.parallelism': '4'}


Result: 59.47 seconds

Testing: Config 3 (Tiny Executors)
Settings: {'spark.executor.memory': '1g', 'spark.executor.cores': '1', 'spark.task.cpus': '1', 'spark.default.parallelism': '24'}


Result: 31.22 seconds

Testing: Config 4 (Balanced 2x2)
Settings: {'spark.executor.memory': '2g', 'spark.executor.cores': '2', 'spark.task.cpus': '1', 'spark.default.parallelism': '16'}


Result: 26.49 seconds

BENCHMARK LEADERBOARD
1. Config 1 (Standard 4x4)   : 25.13s
2. Config 4 (Balanced 2x2)   : 26.49s
3. Config 3 (Tiny Executors) : 31.22s
4. Config 2 (Fat Tasks)      : 59.47s

Initializing session with winner: Config 1 (Standard 4x4)
Ready to proceed with notebook!


## 2. Load Bronze Data and Convert to RDD

We load the Parquet data into a DataFrame first (for efficient reading), then immediately convert to RDD for MapReduce operations.

In [36]:
# Define paths (matching Bronze notebook)
BRONZE_PATH = "../data/medallion/bronze/"
SILVER_PATH = "../data/medallion/silver/"

BRONZE_ACCEPTED_PATH = os.path.join(BRONZE_PATH, "accepted_loans")
BRONZE_REJECTED_PATH = os.path.join(BRONZE_PATH, "rejected_loans")

# Create silver directory
os.makedirs(SILVER_PATH, exist_ok=True)

print(f"Bronze input path: {BRONZE_PATH}")
print(f"Silver output path: {SILVER_PATH}")

Bronze input path: ../data/medallion/bronze/
Silver output path: ../data/medallion/silver/


In [37]:
# Load Bronze data and convert to RDD
# We read parquet (efficient) then convert to RDD of Row objects

accepted_df = spark.read.parquet(BRONZE_ACCEPTED_PATH)
rejected_df = spark.read.parquet(BRONZE_REJECTED_PATH)

# Get column names for reference
accepted_columns = accepted_df.columns
rejected_columns = rejected_df.columns

print(f"Accepted loans columns: {len(accepted_columns)}")
print(f"Rejected loans columns: {len(rejected_columns)}")

# Convert to RDD - each element is a Row object
accepted_rdd = accepted_df.rdd
rejected_rdd = rejected_df.rdd

print(f"\nAccepted loans RDD partitions: {accepted_rdd.getNumPartitions()}")
print(f"Rejected loans RDD partitions: {rejected_rdd.getNumPartitions()}")

Accepted loans columns: 155
Rejected loans columns: 13

Accepted loans RDD partitions: 18
Rejected loans RDD partitions: 18


In [38]:
# Examine sample row structure
sample_row = accepted_rdd.first()
print("Sample row type:", type(sample_row))
print("\nSample row as dict (first 10 fields):")
sample_dict = sample_row.asDict()
for i, (k, v) in enumerate(sample_dict.items()):
    if i < 10:
        print(f"  {k}: {v} (type: {type(v).__name__})")

Sample row type: <class 'pyspark.sql.types.Row'>

Sample row as dict (first 10 fields):
  _data_source: lending_club (type: str)
  _ingestion_timestamp: 1764368685.8879948 (type: float)
  _source_file: accepted_2007_to_2018Q4.csv (type: str)
  _status: valid (type: str)
  acc_now_delinq: 0.0 (type: str)
  acc_open_past_24mths: 4.0 (type: str)
  addr_state: NV (type: str)
  all_util: 70.0 (type: str)
  annual_inc: 84000.0 (type: str)
  annual_inc_joint:  (type: str)


## 3. Identify Data Quality Issues

Before cleaning, let's identify the specific issues in the data using MapReduce operations.

In [39]:
# Select key columns for our use case (loan default prediction)
# These are the columns we'll clean and use in the Gold layer

KEY_COLUMNS = [
    # Loan characteristics
    'loan_amnt',        # Loan amount
    'term',             # Loan term (36 or 60 months)
    'int_rate',         # Interest rate
    'installment',      # Monthly installment
    'grade',            # Loan grade (A-G)
    'sub_grade',        # Loan sub-grade (A1-G5)
    
    # Borrower information
    'emp_length',       # Employment length
    'home_ownership',   # Home ownership status
    'annual_inc',       # Annual income
    'verification_status',  # Income verification
    
    # Loan purpose and status
    'purpose',          # Loan purpose
    'loan_status',      # Current loan status (TARGET)
    'issue_d',          # Issue date
    
    # Credit history
    'dti',              # Debt-to-income ratio
    'earliest_cr_line', # Earliest credit line
    'open_acc',         # Open credit accounts
    'pub_rec',          # Public records
    'revol_bal',        # Revolving balance
    'revol_util',       # Revolving utilization
    'total_acc',        # Total accounts
    'fico_range_low',   # FICO score low
    'fico_range_high',  # FICO score high
    
    # Additional useful features
    'addr_state',       # State
    'delinq_2yrs',      # Delinquencies in 2 years
    'inq_last_6mths',   # Inquiries in last 6 months
    'mort_acc',         # Mortgage accounts
    'pub_rec_bankruptcies'  # Bankruptcies
]

print(f"Selected {len(KEY_COLUMNS)} key columns for cleaning")

Selected 27 key columns for cleaning


In [40]:
%%time
# Profile null values using MapReduce
# Map: emit (column_name, 1) if value is null/empty, else (column_name, 0)
# Reduce: sum to get total nulls per column

def count_nulls_mapper(row):
    """Map function to count nulls for each column"""
    row_dict = row.asDict()
    results = []
    for col in KEY_COLUMNS:
        if col in row_dict:
            value = row_dict[col]
            # Check for null, None, empty string, or 'null' string
            is_null = (value is None or 
                      value == '' or 
                      str(value).lower() == 'null' or
                      str(value).lower() == 'nan')
            results.append((col, 1 if is_null else 0))
    return results

# Use flatMap since mapper returns multiple pairs
null_counts = accepted_rdd \
    .flatMap(count_nulls_mapper) \
    .reduceByKey(lambda a, b: a + b) \
    .collect()

total_rows = accepted_rdd.count()

print(f"Total rows: {total_rows:,}")
print("\nNull/Missing values per column:")
print("-" * 50)
for col, count in sorted(null_counts, key=lambda x: x[1], reverse=True):
    pct = (count / total_rows) * 100
    print(f"{col:25s}: {count:>10,} ({pct:5.2f}%)")

Total rows: 2,260,701

Null/Missing values per column:
--------------------------------------------------
emp_length               :    146,938 ( 6.50%)
mort_acc                 :     53,470 ( 2.37%)
open_acc                 :     44,417 ( 1.96%)
pub_rec_bankruptcies     :     21,252 ( 0.94%)
purpose                  :     21,010 ( 0.93%)
pub_rec                  :     12,308 ( 0.54%)
revol_bal                :      5,805 ( 0.26%)
revol_util               :      4,931 ( 0.22%)
total_acc                :      1,927 ( 0.09%)
dti                      :      1,754 ( 0.08%)
earliest_cr_line         :         57 ( 0.00%)
delinq_2yrs              :         54 ( 0.00%)
inq_last_6mths           :         53 ( 0.00%)
addr_state               :         48 ( 0.00%)
fico_range_low           :         48 ( 0.00%)
home_ownership           :         42 ( 0.00%)
fico_range_high          :         40 ( 0.00%)
annual_inc               :         38 ( 0.00%)
installment              :         33 ( 0.00%)
v

In [41]:
%%time
# Profile unique values for categorical columns using MapReduce

categorical_cols = ['term', 'grade', 'sub_grade', 'emp_length', 
                    'home_ownership', 'verification_status', 'purpose', 'loan_status']

def extract_categorical_mapper(row):
    """Extract categorical column values"""
    row_dict = row.asDict()
    results = []
    for col in categorical_cols:
        if col in row_dict:
            value = str(row_dict[col]) if row_dict[col] is not None else 'NULL'
            results.append(((col, value), 1))
    return results

# Count occurrences of each value per column
categorical_counts = accepted_rdd \
    .flatMap(extract_categorical_mapper) \
    .reduceByKey(lambda a, b: a + b) \
    .collect()

# Organize by column
cat_summary = defaultdict(dict)
for (col, value), count in categorical_counts:
    cat_summary[col][value] = count

# Display
for col in categorical_cols:
    print(f"\n{col}:")
    values = cat_summary[col]
    for value, count in sorted(values.items(), key=lambda x: x[1], reverse=True)[:10]:
        pct = (count / total_rows) * 100
        print(f"  {value:30s}: {count:>10,} ({pct:5.2f}%)")


term:
   36 months                    :  1,609,754 (71.21%)
   60 months                    :    650,914 (28.79%)
                                :         33 ( 0.00%)

grade:
  B                             :    663,557 (29.35%)
  C                             :    650,053 (28.75%)
  A                             :    433,027 (19.15%)
  D                             :    324,424 (14.35%)
  E                             :    135,639 ( 6.00%)
  F                             :     41,800 ( 1.85%)
  G                             :     12,168 ( 0.54%)
                                :         33 ( 0.00%)

sub_grade:
  C1                            :    145,903 ( 6.45%)
  B5                            :    140,288 ( 6.21%)
  B4                            :    139,793 ( 6.18%)
  B3                            :    131,514 ( 5.82%)
  C2                            :    131,116 ( 5.80%)
  C3                            :    129,193 ( 5.71%)
  C4                            :    127,115 ( 5.62%)
 

## 4. Apply Cleaning via MapReduce

This is the core cleaning step using **map** transformation.

In [42]:
def create_loan_status_binary(loan_status):
    """
    Create binary target variable for loan default prediction.
    1 = Default (bad loan)
    0 = Paid/Current (good loan)
    None = Unknown/Exclude
    """
    if loan_status is None:
        return None
    
    status = str(loan_status).lower().strip()
    
    # Bad loans (default = 1)
    bad_statuses = ['charged off', 'default', 'late (31-120 days)', 
                   'late (16-30 days)', 'does not meet the credit policy. status:charged off']
    
    # Good loans (default = 0)
    good_statuses = ['fully paid', 'current', 
                    'does not meet the credit policy. status:fully paid',
                    'in grace period']
    
    if status in bad_statuses:
        return 1
    elif status in good_statuses:
        return 0
    else:
        return None  # Exclude unclear statuses

In [43]:
def clean_accepted_loan_row(row):
    """
    Main cleaning function applied to each row via map().
    Returns a dictionary with cleaned values.
    """
    row_dict = row.asDict()
    
    cleaned = {
        # Loan characteristics
        'loan_amnt': clean_numeric(row_dict.get('loan_amnt')),
        'term': clean_term(row_dict.get('term')),
        'int_rate': clean_percentage(row_dict.get('int_rate')) if '%' in str(row_dict.get('int_rate', '')) else clean_numeric(row_dict.get('int_rate')),
        'installment': clean_numeric(row_dict.get('installment')),
        'grade': clean_string(row_dict.get('grade')),
        'sub_grade': clean_string(row_dict.get('sub_grade')),
        
        # Borrower information
        'emp_length': clean_emp_length(row_dict.get('emp_length')),
        'home_ownership': clean_string(row_dict.get('home_ownership')),
        'annual_inc': clean_numeric(row_dict.get('annual_inc')),
        'verification_status': clean_string(row_dict.get('verification_status')),
        
        # Loan purpose and status
        'purpose': clean_string(row_dict.get('purpose')),
        'loan_status': clean_string(row_dict.get('loan_status')),
        'loan_status_binary': create_loan_status_binary(row_dict.get('loan_status')),
        'issue_d': clean_date(row_dict.get('issue_d')),
        
        # Credit history
        'dti': clean_numeric(row_dict.get('dti')),
        'earliest_cr_line': clean_date(row_dict.get('earliest_cr_line')),
        'open_acc': clean_numeric(row_dict.get('open_acc')),
        'pub_rec': clean_numeric(row_dict.get('pub_rec')),
        'revol_bal': clean_numeric(row_dict.get('revol_bal')),
        'revol_util': clean_percentage(row_dict.get('revol_util')) if '%' in str(row_dict.get('revol_util', '')) else clean_numeric(row_dict.get('revol_util')),
        'total_acc': clean_numeric(row_dict.get('total_acc')),
        'fico_range_low': clean_numeric(row_dict.get('fico_range_low')),
        'fico_range_high': clean_numeric(row_dict.get('fico_range_high')),
        
        # Additional features
        'addr_state': clean_string(row_dict.get('addr_state')),
        'delinq_2yrs': clean_numeric(row_dict.get('delinq_2yrs')),
        'inq_last_6mths': clean_numeric(row_dict.get('inq_last_6mths')),
        'mort_acc': clean_numeric(row_dict.get('mort_acc')),
        'pub_rec_bankruptcies': clean_numeric(row_dict.get('pub_rec_bankruptcies')),
    }
    
    # Calculate derived features
    # FICO average
    if cleaned['fico_range_low'] and cleaned['fico_range_high']:
        cleaned['fico_avg'] = (cleaned['fico_range_low'] + cleaned['fico_range_high']) / 2
    else:
        cleaned['fico_avg'] = None
    
    # Loan to income ratio
    if cleaned['loan_amnt'] and cleaned['annual_inc'] and cleaned['annual_inc'] > 0:
        cleaned['loan_to_income'] = cleaned['loan_amnt'] / cleaned['annual_inc']
    else:
        cleaned['loan_to_income'] = None
    
    return cleaned

# Test on sample row
test_cleaned = clean_accepted_loan_row(sample_row)
print("Cleaned sample row:")
for k, v in test_cleaned.items():
    print(f"  {k}: {v}")

Cleaned sample row:
  loan_amnt: 11200.0
  term: 60
  int_rate: 15.04
  installment: 266.69
  grade: C
  sub_grade: C4
  emp_length: 5
  home_ownership: MORTGAGE
  annual_inc: 84000.0
  verification_status: Not Verified
  purpose: debt_consolidation
  loan_status: Current
  loan_status_binary: 0
  issue_d: 2018-04-01
  dti: 22.1
  earliest_cr_line: 2002-02-01
  open_acc: 13.0
  pub_rec: 0.0
  revol_bal: 33431.0
  revol_util: 62.1
  total_acc: 16.0
  fico_range_low: 725.0
  fico_range_high: 729.0
  addr_state: NV
  delinq_2yrs: 0.0
  inq_last_6mths: 0.0
  mort_acc: 1.0
  pub_rec_bankruptcies: 0.0
  fico_avg: 727.0
  loan_to_income: 0.13333333333333333


In [44]:
%%time
# Apply cleaning transformation using map()
# This is the core MapReduce cleaning operation

print("Applying cleaning transformation via map()...")

cleaned_rdd = accepted_rdd.map(clean_accepted_loan_row)

# Cache for reuse (important for performance)
cleaned_rdd.cache()

# Force evaluation and count
cleaned_count = cleaned_rdd.count()
print(f"Cleaned RDD count: {cleaned_count:,}")

Applying cleaning transformation via map()...


Cleaned RDD count: 2,260,701
CPU times: user 21.4 ms, sys: 8.5 ms, total: 29.9 ms
Wall time: 21.7 s


## 5. Filter Invalid Records

Using **filter** transformation to remove invalid records.

In [45]:
def is_valid_loan_record(row_dict):
    """
    Filter function to identify valid records.
    Returns True if record should be kept.
    """
    # Must have loan amount
    if row_dict.get('loan_amnt') is None or row_dict['loan_amnt'] <= 0:
        return False
    
    # Must have a determinable loan status for ML
    if row_dict.get('loan_status_binary') is None:
        return False
    
    # Must have interest rate
    if row_dict.get('int_rate') is None or row_dict['int_rate'] <= 0:
        return False
    
    # Must have grade
    if row_dict.get('grade') is None:
        return False
    
    # Must have annual income (and it should be positive)
    if row_dict.get('annual_inc') is None or row_dict['annual_inc'] <= 0:
        return False
    
    # Reasonable bounds check
    # Interest rate should be between 0 and 50%
    if row_dict['int_rate'] > 50:
        return False
    
    # Annual income should be reasonable (< $10M)
    if row_dict['annual_inc'] > 10000000:
        return False
    
    return True

In [46]:
%%time
# Apply filter transformation
print("Applying filter transformation...")

filtered_rdd = cleaned_rdd.filter(is_valid_loan_record)

# Cache filtered RDD
filtered_rdd.cache()

filtered_count = filtered_rdd.count()
removed_count = cleaned_count - filtered_count

print(f"Records after filtering: {filtered_count:,}")
print(f"Records removed: {removed_count:,} ({removed_count/cleaned_count*100:.2f}%)")

Applying filter transformation...


Records after filtering: 2,231,965
Records removed: 28,736 (1.27%)
CPU times: user 13.7 ms, sys: 3.64 ms, total: 17.3 ms
Wall time: 1.77 s


## 6. Profile Cleaned Data with MapReduce Aggregations

Using **reduceByKey**, **aggregateByKey**, and other MapReduce operations to profile the cleaned data.

In [47]:
%%time
# Compute statistics using MapReduce
# Count by loan_status_binary using map + reduceByKey

status_counts = filtered_rdd \
    .map(lambda x: (x['loan_status_binary'], 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .collect()

print("Loan Status Distribution:")
for status, count in sorted(status_counts):
    label = "Default" if status == 1 else "Paid/Current"
    pct = count / filtered_count * 100
    print(f"  {label} ({status}): {count:>10,} ({pct:5.2f}%)")

Loan Status Distribution:
  Paid/Current (0):  1,939,597 (86.90%)
  Default (1):    292,368 (13.10%)
CPU times: user 13.3 ms, sys: 2.53 ms, total: 15.9 ms
Wall time: 844 ms


In [48]:
%%time
# Grade distribution using MapReduce

grade_counts = filtered_rdd \
    .map(lambda x: (x['grade'], 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .collect()

print("Grade Distribution:")
for grade, count in sorted(grade_counts):
    pct = count / filtered_count * 100
    print(f"  Grade {grade}: {count:>10,} ({pct:5.2f}%)")

Grade Distribution:
  Grade A:    426,139 (19.09%)
  Grade B:    654,756 (29.34%)
  Grade C:    642,765 (28.80%)
  Grade D:    320,918 (14.38%)
  Grade E:    134,102 ( 6.01%)
  Grade F:     41,272 ( 1.85%)
  Grade G:     12,013 ( 0.54%)
CPU times: user 10.5 ms, sys: 6.34 ms, total: 16.8 ms
Wall time: 923 ms


In [49]:
%%time
# Default rate by grade using MapReduce
# Map: (grade, (default_status, 1))
# Reduce: (grade, (total_defaults, total_count))

def grade_default_mapper(row):
    grade = row['grade']
    is_default = row['loan_status_binary']
    return (grade, (is_default, 1))

def grade_default_reducer(a, b):
    return (a[0] + b[0], a[1] + b[1])

default_by_grade = filtered_rdd \
    .map(grade_default_mapper) \
    .reduceByKey(grade_default_reducer) \
    .collect()

print("Default Rate by Grade:")
print("-" * 50)
for grade, (defaults, total) in sorted(default_by_grade):
    rate = defaults / total * 100
    print(f"  Grade {grade}: {rate:5.2f}% default rate ({defaults:,} / {total:,})")

Default Rate by Grade:
--------------------------------------------------
  Grade A:  3.68% default rate (15,691 / 426,139)
  Grade B:  8.83% default rate (57,843 / 654,756)
  Grade C: 14.62% default rate (93,971 / 642,765)
  Grade D: 20.69% default rate (66,389 / 320,918)
  Grade E: 28.65% default rate (38,417 / 134,102)
  Grade F: 36.83% default rate (15,200 / 41,272)
  Grade G: 40.43% default rate (4,857 / 12,013)
CPU times: user 12.3 ms, sys: 2.85 ms, total: 15.1 ms
Wall time: 970 ms


In [50]:
%%time
# Compute basic statistics for numeric columns using aggregate
# Using aggregate to compute min, max, sum, count in one pass

def stats_seq_op(acc, row):
    """Sequential operation: update accumulator with new row"""
    value = row.get('loan_amnt')
    if value is not None:
        min_val = min(acc[0], value) if acc[0] is not None else value
        max_val = max(acc[1], value) if acc[1] is not None else value
        sum_val = acc[2] + value
        count_val = acc[3] + 1
        return (min_val, max_val, sum_val, count_val)
    return acc

def stats_comb_op(acc1, acc2):
    """Combine operation: merge two accumulators"""
    min_val = min(acc1[0], acc2[0]) if acc1[0] is not None and acc2[0] is not None else acc1[0] or acc2[0]
    max_val = max(acc1[1], acc2[1]) if acc1[1] is not None and acc2[1] is not None else acc1[1] or acc2[1]
    sum_val = acc1[2] + acc2[2]
    count_val = acc1[3] + acc2[3]
    return (min_val, max_val, sum_val, count_val)

# Initial accumulator: (min, max, sum, count)
zero_value = (None, None, 0.0, 0)

loan_amnt_stats = filtered_rdd.aggregate(zero_value, stats_seq_op, stats_comb_op)

min_val, max_val, sum_val, count_val = loan_amnt_stats
avg_val = sum_val / count_val if count_val > 0 else 0

print("Loan Amount Statistics:")
print(f"  Min: ${min_val:,.2f}")
print(f"  Max: ${max_val:,.2f}")
print(f"  Avg: ${avg_val:,.2f}")
print(f"  Count: {count_val:,}")

Loan Amount Statistics:
  Min: $500.00
  Max: $40,000.00
  Avg: $15,010.02
  Count: 2,231,965
CPU times: user 5.42 ms, sys: 4.9 ms, total: 10.3 ms
Wall time: 689 ms


In [51]:
%%time
# Compute stats for multiple columns using a single pass with aggregate

numeric_cols = ['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'fico_avg']

def multi_stats_seq_op(acc, row):
    """Update stats for all numeric columns"""
    result = dict(acc)
    for col in numeric_cols:
        value = row.get(col)
        if value is not None:
            stats = result[col]
            min_val = min(stats[0], value) if stats[0] is not None else value
            max_val = max(stats[1], value) if stats[1] is not None else value
            sum_val = stats[2] + value
            count_val = stats[3] + 1
            result[col] = (min_val, max_val, sum_val, count_val)
    return result

def multi_stats_comb_op(acc1, acc2):
    """Combine stats from two accumulators"""
    result = {}
    for col in numeric_cols:
        s1, s2 = acc1[col], acc2[col]
        min_val = min(s1[0], s2[0]) if s1[0] is not None and s2[0] is not None else s1[0] or s2[0]
        max_val = max(s1[1], s2[1]) if s1[1] is not None and s2[1] is not None else s1[1] or s2[1]
        sum_val = s1[2] + s2[2]
        count_val = s1[3] + s2[3]
        result[col] = (min_val, max_val, sum_val, count_val)
    return result

zero_multi = {col: (None, None, 0.0, 0) for col in numeric_cols}

multi_stats = filtered_rdd.aggregate(zero_multi, multi_stats_seq_op, multi_stats_comb_op)

print("Numeric Column Statistics:")
print("=" * 70)
print(f"{'Column':<15} {'Min':>12} {'Max':>12} {'Avg':>12} {'Count':>12}")
print("-" * 70)
for col in numeric_cols:
    min_v, max_v, sum_v, count_v = multi_stats[col]
    avg_v = sum_v / count_v if count_v > 0 else 0
    print(f"{col:<15} {min_v:>12,.2f} {max_v:>12,.2f} {avg_v:>12,.2f} {count_v:>12,}")

Numeric Column Statistics:
Column                   Min          Max          Avg        Count
----------------------------------------------------------------------
loan_amnt             500.00    40,000.00    15,010.02    2,231,965
int_rate                5.31        30.99        13.10    2,231,965
annual_inc              0.36 9,930,475.00    77,601.46    2,231,965
dti                    -1.00     2,800.00        18.91    2,194,254
fico_avg                0.72     2,271.12       700.27    2,194,763
CPU times: user 6.62 ms, sys: 4.06 ms, total: 10.7 ms
Wall time: 962 ms


## 7. Performance Profiling and Tuning

Let's profile and tune our MapReduce operations.

In [52]:
print("=== Partition Analysis ===")
print(f"Original RDD partitions: {accepted_rdd.getNumPartitions()}")
print(f"Cleaned RDD partitions: {cleaned_rdd.getNumPartitions()}")
print(f"Filtered RDD partitions: {filtered_rdd.getNumPartitions()}")

print("\n--- Detailed Partition Analysis (using glom) ---")

# Use glom() to get the list of rows per partition
# Then map(len) to just calculate the size of that list
# This runs entirely on the workers, sending only 18 integers back to the driver
partition_sizes = filtered_rdd.glom().map(len).collect()

print(f"Total partitions: {len(partition_sizes)}")
print(f"Total records: {sum(partition_sizes):,}")
print(f"Min partition size: {min(partition_sizes):,}")
print(f"Max partition size: {max(partition_sizes):,}")
print(f"Avg partition size: {sum(partition_sizes)/len(partition_sizes):,.0f}")

# Check for Skew
skew_ratio = max(partition_sizes) / min(partition_sizes) if min(partition_sizes) > 0 else 0
print(f"Skew Ratio (Max/Min): {skew_ratio:.2f}x")

if skew_ratio > 1.5:
    print("Data is skewed! Some partitions are much larger than others.")
else:
    print("Data is well balanced across partitions.")

print(f"\nRaw partition sizes: {partition_sizes}")

=== Partition Analysis ===
Original RDD partitions: 18
Cleaned RDD partitions: 18
Filtered RDD partitions: 18

--- Detailed Partition Analysis (using glom) ---


Total partitions: 18
Total records: 2,231,965
Min partition size: 43,842
Max partition size: 143,010
Avg partition size: 123,998
Skew Ratio (Max/Min): 3.26x
Data is skewed! Some partitions are much larger than others.

Raw partition sizes: [81597, 82794, 131248, 132285, 132125, 133400, 133577, 132672, 133310, 131546, 136336, 134565, 135174, 142867, 143010, 139136, 132481, 43842]


In [53]:
%%time
# Experiment with different partition counts
# Find optimal partition count based on data size
# For ~30M records, let's try different values

import time

partition_tests = [2, 4, 8, 16, 32, 64]
results = []

for num_partitions in partition_tests:
    # Repartition
    test_rdd = filtered_rdd.repartition(num_partitions)
    
    # Time a simple operation
    start = time.time()
    
    # Perform a MapReduce operation
    _ = test_rdd \
        .map(lambda x: (x['grade'], x['loan_amnt'])) \
        .reduceByKey(lambda a, b: a + b) \
        .collect()
    
    elapsed = time.time() - start
    results.append((num_partitions, elapsed))
    print(f"Partitions: {num_partitions:2d} | Time: {elapsed:.3f}s")

# Find best
best = min(results, key=lambda x: x[1])
print(f"\nOptimal partition count: {best[0]} (time: {best[1]:.3f}s)")

Partitions:  2 | Time: 4.024s


Partitions:  4 | Time: 2.829s


Partitions:  8 | Time: 2.309s


Partitions: 16 | Time: 2.083s


Partitions: 32 | Time: 2.290s


Partitions: 64 | Time: 2.418s

Optimal partition count: 16 (time: 2.083s)
CPU times: user 83.2 ms, sys: 40 ms, total: 123 ms
Wall time: 16 s


In [54]:
# Apply optimal partitioning
OPTIMAL_PARTITIONS = 16  # Adjust based on results above

optimized_rdd = filtered_rdd.repartition(OPTIMAL_PARTITIONS)
optimized_rdd.cache()

# Force evaluation
_ = optimized_rdd.count()

print(f"Optimized RDD partitions: {optimized_rdd.getNumPartitions()}")

Optimized RDD partitions: 16


## 8. Process Rejected Loans

Apply similar cleaning to rejected loans dataset.

In [55]:
# Check rejected loans structure
rejected_sample = rejected_rdd.first()
print("Rejected loans columns:")
for k, v in rejected_sample.asDict().items():
    print(f"  {k}: {v}")

Rejected loans columns:
  Amount Requested: 1000.0
  Application Date: 2007-05-26
  Debt-To-Income Ratio: 10%
  Employment Length: 4 years
  Loan Title: Wedding Covered but No Honeymoon
  Policy Code: 0.0
  Risk_Score: 693.0
  State: NM
  Zip Code: 481xx
  _data_source: lending_club
  _ingestion_timestamp: 1764368694.9139888
  _source_file: rejected_2007_to_2018Q4.csv
  _status: valid


In [56]:
def clean_rejected_loan_row(row):
    """
    Clean rejected loan records using map().
    """
    row_dict = row.asDict()
    
    cleaned = {
        'Amount Requested': clean_numeric(row_dict.get('Amount Requested')),
        'Application Date': clean_string(row_dict.get('Application Date')),
        'Loan Title': clean_string(row_dict.get('Loan Title')),
        'Risk_Score': clean_numeric(row_dict.get('Risk_Score')),
        'Debt-To-Income Ratio': clean_percentage(row_dict.get('Debt-To-Income Ratio')),
        'Zip Code': clean_string(row_dict.get('Zip Code')),
        'State': clean_string(row_dict.get('State')),
        'Employment Length': clean_emp_length(row_dict.get('Employment Length')),
        'Policy Code': clean_string(row_dict.get('Policy Code')),
    }
    
    return cleaned

def is_valid_rejected_record(row_dict):
    """
    Filter valid rejected records with comprehensive data quality checks.
    Same criteria as Project.ipynb:
    - loan_amount: not null, > 0, <= 100,000
    - debt_to_income_ratio: not null, >= 0, <= 100
    - risk_score: null OR (>= 300 AND <= 850)
    """
    # Check loan amount
    loan_amt = row_dict.get('Amount Requested')
    if loan_amt is None or loan_amt <= 0 or loan_amt > 100000:
        return False
    
    # Check debt-to-income ratio (must not be null and within range)
    dti = row_dict.get('Debt-To-Income Ratio')
    if dti is None or dti < 0 or dti > 100:
        return False
    
    # Check risk score (can be null, but if present must be in valid range)
    risk_score = row_dict.get('Risk_Score')
    if risk_score is not None and (risk_score < 300 or risk_score > 850):
        return False
    
    return True

In [57]:
%%time
# Clean rejected loans using MapReduce with comprehensive data quality checks

print("Cleaning rejected loans...")
print("Applying data quality filters:")
print("  1. Basic cleaning (percentages, strings, employment length)")
print("  2. Unrealistic value filtering (amount, DTI, risk_score)")
print("  3. Date validation (2007-2018)")
print("  4. Deduplication")

# Step 1: Clean using map()
cleaned_rejected_rdd = rejected_rdd.map(clean_rejected_loan_row)

# Step 2: Filter unrealistic values using filter()
initial_count = cleaned_rejected_rdd.count()
print(f"\nInitial records: {initial_count:,}")

valid_rejected_rdd = cleaned_rejected_rdd.filter(is_valid_rejected_record)
after_validation_count = valid_rejected_rdd.count()
removed_validation = initial_count - after_validation_count
print(f"After validation filters: {after_validation_count:,} (removed {removed_validation:,})")

# Step 3: Date validation using filter()
def is_valid_date_range(row_dict):
    """Check if application date is within 2007-2018"""
    app_date = row_dict.get('Application Date')
    if app_date is None:
        return True  # Allow null dates
    
    try:
        # Parse date format: "yyyy-MM-dd"
        from datetime import datetime
        date_obj = datetime.strptime(app_date, '%Y-%m-%d')
        year = date_obj.year
        return 2007 <= year <= 2018
    except:
        return False  # Invalid date format

date_filtered_rdd = valid_rejected_rdd.filter(is_valid_date_range)
after_date_count = date_filtered_rdd.count()
removed_dates = after_validation_count - after_date_count
print(f"After date validation: {after_date_count:,} (removed {removed_dates:,})")

# Step 4: Deduplication using MapReduce pattern
# Create composite key from key fields, then use reduceByKey to keep first occurrence
def create_dedup_key(row_dict):
    """Create composite key for deduplication"""
    key = (
        row_dict.get('Amount Requested'),
        row_dict.get('Loan Title'),
        row_dict.get('Application Date'),
        row_dict.get('State'),
        row_dict.get('Zip Code')
    )
    return (key, row_dict)

# Map to (key, row), reduceByKey to keep first, then extract row
deduped_rdd = date_filtered_rdd \
    .map(create_dedup_key) \
    .reduceByKey(lambda a, b: a) \
    .map(lambda x: x[1])

deduped_rdd.cache()

final_rejected_count = deduped_rdd.count()
removed_duplicates = after_date_count - final_rejected_count
print(f"After deduplication: {final_rejected_count:,} (removed {removed_duplicates:,})")

print(f"\n Total removed: {initial_count - final_rejected_count:,} ({100*(initial_count - final_rejected_count)/initial_count:.2f}%)")

# Update reference to use deduplicated RDD
cleaned_rejected_rdd = deduped_rdd

Cleaning rejected loans...
Applying data quality filters:
  1. Basic cleaning (percentages, strings, employment length)
  2. Unrealistic value filtering (amount, DTI, risk_score)
  3. Date validation (2007-2018)
  4. Deduplication



Initial records: 27,648,741


After validation filters: 25,528,260 (removed 2,120,481)


After date validation: 25,528,260 (removed 0)


25/11/28 22:40:02 WARN HeartbeatReceiver: Removing executor 1 with no recent heartbeats: 129923 ms exceeds timeout 120000 ms
25/11/28 22:40:02 WARN HeartbeatReceiver: Removing executor 0 with no recent heartbeats: 125104 ms exceeds timeout 120000 ms
25/11/28 22:40:02 ERROR TaskSchedulerImpl: Lost executor 1 on 192.168.18.110: Executor heartbeat timed out after 129923 ms
25/11/28 22:40:02 WARN TaskSetManager: Lost task 8.0 in stage 43.0 (TID 708) (192.168.18.110 executor 1): ExecutorLostFailure (executor 1 exited caused by one of the running tasks) Reason: Executor heartbeat timed out after 129923 ms
25/11/28 22:40:02 WARN TaskSetManager: Lost task 4.0 in stage 43.0 (TID 704) (192.168.18.110 executor 1): ExecutorLostFailure (executor 1 exited caused by one of the running tasks) Reason: Executor heartbeat timed out after 129923 ms
25/11/28 22:40:02 WARN TaskSetManager: Lost task 0.0 in stage 43.0 (TID 700) (192.168.18.110 executor 1): ExecutorLostFailure (executor 1 exited caused by one 

After deduplication: 20,116,218 (removed 5,412,042)

 Total removed: 7,532,523 (27.24%)
CPU times: user 447 ms, sys: 1min 7s, total: 1min 7s
Wall time: 9min 49s


## 9. Save to Silver Layer

Convert cleaned RDDs to DataFrames and save as Parquet.

In [58]:
# Define schema for accepted loans Silver layer
accepted_silver_schema = StructType([
    StructField("loan_amnt", FloatType(), True),
    StructField("term", IntegerType(), True),
    StructField("int_rate", FloatType(), True),
    StructField("installment", FloatType(), True),
    StructField("grade", StringType(), True),
    StructField("sub_grade", StringType(), True),
    StructField("emp_length", IntegerType(), True),
    StructField("home_ownership", StringType(), True),
    StructField("annual_inc", FloatType(), True),
    StructField("verification_status", StringType(), True),
    StructField("purpose", StringType(), True),
    StructField("loan_status", StringType(), True),
    StructField("loan_status_binary", IntegerType(), True),
    StructField("issue_d", StringType(), True),
    StructField("dti", FloatType(), True),
    StructField("earliest_cr_line", StringType(), True),
    StructField("open_acc", FloatType(), True),
    StructField("pub_rec", FloatType(), True),
    StructField("revol_bal", FloatType(), True),
    StructField("revol_util", FloatType(), True),
    StructField("total_acc", FloatType(), True),
    StructField("fico_range_low", FloatType(), True),
    StructField("fico_range_high", FloatType(), True),
    StructField("addr_state", StringType(), True),
    StructField("delinq_2yrs", FloatType(), True),
    StructField("inq_last_6mths", FloatType(), True),
    StructField("mort_acc", FloatType(), True),
    StructField("pub_rec_bankruptcies", FloatType(), True),
    StructField("fico_avg", FloatType(), True),
    StructField("loan_to_income", FloatType(), True),
])

In [59]:
# Convert RDD of dicts to RDD of Rows
from pyspark.sql import Row

def dict_to_row(d):
    """Convert dictionary to Row with proper type handling"""
    return Row(
        loan_amnt=float(d['loan_amnt']) if d['loan_amnt'] is not None else None,
        term=int(d['term']) if d['term'] is not None else None,
        int_rate=float(d['int_rate']) if d['int_rate'] is not None else None,
        installment=float(d['installment']) if d['installment'] is not None else None,
        grade=d['grade'],
        sub_grade=d['sub_grade'],
        emp_length=int(d['emp_length']) if d['emp_length'] is not None else None,
        home_ownership=d['home_ownership'],
        annual_inc=float(d['annual_inc']) if d['annual_inc'] is not None else None,
        verification_status=d['verification_status'],
        purpose=d['purpose'],
        loan_status=d['loan_status'],
        loan_status_binary=int(d['loan_status_binary']) if d['loan_status_binary'] is not None else None,
        issue_d=d['issue_d'],
        dti=float(d['dti']) if d['dti'] is not None else None,
        earliest_cr_line=d['earliest_cr_line'],
        open_acc=float(d['open_acc']) if d['open_acc'] is not None else None,
        pub_rec=float(d['pub_rec']) if d['pub_rec'] is not None else None,
        revol_bal=float(d['revol_bal']) if d['revol_bal'] is not None else None,
        revol_util=float(d['revol_util']) if d['revol_util'] is not None else None,
        total_acc=float(d['total_acc']) if d['total_acc'] is not None else None,
        fico_range_low=float(d['fico_range_low']) if d['fico_range_low'] is not None else None,
        fico_range_high=float(d['fico_range_high']) if d['fico_range_high'] is not None else None,
        addr_state=d['addr_state'],
        delinq_2yrs=float(d['delinq_2yrs']) if d['delinq_2yrs'] is not None else None,
        inq_last_6mths=float(d['inq_last_6mths']) if d['inq_last_6mths'] is not None else None,
        mort_acc=float(d['mort_acc']) if d['mort_acc'] is not None else None,
        pub_rec_bankruptcies=float(d['pub_rec_bankruptcies']) if d['pub_rec_bankruptcies'] is not None else None,
        fico_avg=float(d['fico_avg']) if d['fico_avg'] is not None else None,
        loan_to_income=float(d['loan_to_income']) if d['loan_to_income'] is not None else None,
    )

# Transform using map
row_rdd = optimized_rdd.map(dict_to_row)

# Create DataFrame
accepted_silver_df = spark.createDataFrame(row_rdd, schema=accepted_silver_schema)

print(f"Silver DataFrame created with {len(accepted_silver_df.columns)} columns")
accepted_silver_df.printSchema()

Silver DataFrame created with 30 columns
root
 |-- loan_amnt: float (nullable = true)
 |-- term: integer (nullable = true)
 |-- int_rate: float (nullable = true)
 |-- installment: float (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_length: integer (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: float (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- loan_status_binary: integer (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- dti: float (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- open_acc: float (nullable = true)
 |-- pub_rec: float (nullable = true)
 |-- revol_bal: float (nullable = true)
 |-- revol_util: float (nullable = true)
 |-- total_acc: float (nullable = true)
 |-- fico_range_low: float (nullable = true)
 |-- fico_range_high: float (nullable = 

In [60]:
# Preview silver data
accepted_silver_df.select(
    'loan_amnt', 'int_rate', 'grade', 'annual_inc', 
    'loan_status_binary', 'fico_avg', 'loan_to_income'
).show(10, truncate=False)

+---------+--------+-----+----------+------------------+--------+--------------+
|loan_amnt|int_rate|grade|annual_inc|loan_status_binary|fico_avg|loan_to_income|
+---------+--------+-----+----------+------------------+--------+--------------+
|10000.0  |5.32    |A    |170000.0  |0                 |732.0   |0.05882353    |
|11200.0  |11.49   |B    |34000.0   |1                 |712.0   |0.32941177    |
|20000.0  |12.74   |C    |80000.0   |0                 |662.0   |0.25          |
|1500.0   |10.49   |B    |20000.0   |0                 |732.0   |0.075         |
|4800.0   |8.24    |B    |62000.0   |0                 |777.0   |0.077419356   |
|17500.0  |10.49   |B    |80000.0   |0                 |697.0   |0.21875       |
|29175.0  |15.99   |C    |150000.0  |0                 |692.0   |0.1945        |
|10000.0  |13.99   |C    |52000.0   |0                 |677.0   |0.1923077     |
|19200.0  |14.99   |C    |68000.0   |0                 |662.0   |0.28235295    |
|3500.0   |13.49   |C    |40

In [61]:
%%time
import shutil

# Helper function to clean directory before saving
def clean_output_directory(path):
    """Remove existing directory to prevent duplicate files."""
    if os.path.exists(path):
        print(f"Removing existing directory: {path}")
        shutil.rmtree(path)
        print(f"Directory cleaned.")

# Save to Silver layer
SILVER_ACCEPTED_PATH = os.path.join(SILVER_PATH, "accepted_loans")

print("=== Saving Accepted Loans to Silver ===")
clean_output_directory(SILVER_ACCEPTED_PATH)

accepted_silver_df.write \
    .mode("overwrite") \
    .parquet(SILVER_ACCEPTED_PATH)

print(f"Silver data saved to: {SILVER_ACCEPTED_PATH}")

=== Saving Accepted Loans to Silver ===
Removing existing directory: ../data/medallion/silver/accepted_loans
Directory cleaned.


Silver data saved to: ../data/medallion/silver/accepted_loans
CPU times: user 6.24 ms, sys: 17.2 ms, total: 23.5 ms
Wall time: 8.49 s


In [62]:
%%time
# Save rejected loans Silver layer
SILVER_REJECTED_PATH = os.path.join(SILVER_PATH, "rejected_loans")

print("\n=== Saving Rejected Loans to Silver ===")
clean_output_directory(SILVER_REJECTED_PATH)

rejected_silver_schema = StructType([
    StructField("amount_requested", FloatType(), True),
    StructField("application_date", StringType(), True),
    StructField("loan_title", StringType(), True),
    StructField("risk_score", FloatType(), True),
    StructField("dti", FloatType(), True),
    StructField("zip_code", StringType(), True),
    StructField("state", StringType(), True),
    StructField("emp_length", IntegerType(), True),
    StructField("policy_code", StringType(), True),
])

def rejected_dict_to_row(d):
    return Row(
        amount_requested=float(d['Amount Requested']) if d['Amount Requested'] is not None else None,
        application_date=d['Application Date'],
        loan_title=d['Loan Title'],
        risk_score=float(d['Risk_Score']) if d['Risk_Score'] is not None else None,
        dti=float(d['Debt-To-Income Ratio']) if d['Debt-To-Income Ratio'] is not None else None,
        zip_code=d['Zip Code'],
        state=d['State'],
        emp_length=int(d['Employment Length']) if d['Employment Length'] is not None else None,
        policy_code=d['Policy Code'],
    )

rejected_row_rdd = cleaned_rejected_rdd.map(rejected_dict_to_row)
rejected_silver_df = spark.createDataFrame(rejected_row_rdd, schema=rejected_silver_schema)

rejected_silver_df.write \
    .mode("overwrite") \
    .parquet(SILVER_REJECTED_PATH)

print(f"Rejected Silver data saved to: {SILVER_REJECTED_PATH}")


=== Saving Rejected Loans to Silver ===
Removing existing directory: ../data/medallion/silver/rejected_loans
Directory cleaned.


Rejected Silver data saved to: ../data/medallion/silver/rejected_loans
CPU times: user 10.5 ms, sys: 14.2 ms, total: 24.7 ms
Wall time: 11.5 s


## 10. Verification and Summary

In [63]:
# Verify saved data
verify_accepted = spark.read.parquet(SILVER_ACCEPTED_PATH)
verify_rejected = spark.read.parquet(SILVER_REJECTED_PATH)

print("=== Silver Layer Verification ===")
print(f"Accepted loans: {verify_accepted.count():,} rows, {len(verify_accepted.columns)} columns")
print(f"Rejected loans: {verify_rejected.count():,} rows, {len(verify_rejected.columns)} columns")

=== Silver Layer Verification ===
Accepted loans: 2,231,965 rows, 30 columns
Rejected loans: 20,116,218 rows, 9 columns


In [64]:
# Check storage sizes
!du -sh {SILVER_ACCEPTED_PATH}
!du -sh {SILVER_REJECTED_PATH}

70M	../data/medallion/silver/accepted_loans
161M	../data/medallion/silver/rejected_loans


In [65]:
# Final summary statistics (replace existing summary block)
from datetime import datetime

print("=" * 70)
print("SILVER LAYER CLEANING SUMMARY")
print("=" * 70)

# Compute counts once to avoid repeated evaluation
accepted_input = accepted_rdd.count()
accepted_output = verify_accepted.count()
accepted_removed = accepted_input - accepted_output

rejected_input = rejected_rdd.count()
rejected_output = verify_rejected.count()
rejected_removed = rejected_input - rejected_output

print("\nMapReduce Operations Used:")
ops = [
    "map(): Data cleaning and transformation",
    "filter(): Remove invalid records",
    "flatMap(): Profile null values across columns",
    "reduceByKey(): Aggregate statistics & Deduplication",
    "aggregate(): Compute min/max/sum/count in single pass",
    "glom(): Analyze partition distribution",
    "repartition(): Optimize partitioning",
]
for o in ops:
    print("  -", o)

print("\nData Quality Improvements:")
improvements = [
    f"Cleaned {len(KEY_COLUMNS)} key columns",
    "Standardized date formats",
    "Converted percentages and currency values",
    "Sanitized empty strings/whitespace (Null handling)",
    "Created binary target variable for ML",
    "Added derived features (fico_avg, loan_to_income)",
]
for i in improvements:
    print("  -", i)

print("\nAccepted Loans (Cleaned):")
print(f"  - Bronze Input: {accepted_input:,}")
print(f"  - Silver Output: {accepted_output:,}")
print(f"  - Records removed: {accepted_removed:,} ({accepted_removed/accepted_input*100 if accepted_input>0 else 0:.2f}%)")

print("\nRejected Loans (Cleaned & Deduplicated):")
print(f"  - Bronze Input: {rejected_input:,}")
print(f"  - Silver Output: {rejected_output:,}")
print(f"  - Records removed: {rejected_removed:,} ({rejected_removed/rejected_input*100 if rejected_input>0 else 0:.2f}%)")

# Partitioning / performance summary (if available)
try:
    parts = optimized_rdd.getNumPartitions()
    print("\nPartitioning:")
    print(f"  - Optimized RDD partitions: {parts}")
    if 'partition_sizes' in globals() and partition_sizes:
        print(f"  - Partition sizes (min/max/avg): {min(partition_sizes):,}/{max(partition_sizes):,}/{sum(partition_sizes)/len(partition_sizes):.0f}")
except Exception:
    pass

print("\nOutput Paths:")
print(f"  - {SILVER_ACCEPTED_PATH}")
print(f"  - {SILVER_REJECTED_PATH}")

print("\nSummary generated at:", datetime.utcnow().isoformat())
print("=" * 70)

SILVER LAYER CLEANING SUMMARY



MapReduce Operations Used:
  - map(): Data cleaning and transformation
  - filter(): Remove invalid records
  - flatMap(): Profile null values across columns
  - reduceByKey(): Aggregate statistics & Deduplication
  - aggregate(): Compute min/max/sum/count in single pass
  - glom(): Analyze partition distribution
  - repartition(): Optimize partitioning

Data Quality Improvements:
  - Cleaned 27 key columns
  - Standardized date formats
  - Converted percentages and currency values
  - Sanitized empty strings/whitespace (Null handling)
  - Created binary target variable for ML
  - Added derived features (fico_avg, loan_to_income)

Accepted Loans (Cleaned):
  - Bronze Input: 2,260,701
  - Silver Output: 2,231,965
  - Records removed: 28,736 (1.27%)

Rejected Loans (Cleaned & Deduplicated):
  - Bronze Input: 27,648,741
  - Silver Output: 20,116,218
  - Records removed: 7,532,523 (27.24%)

Partitioning:
  - Optimized RDD partitions: 16
  - Partition sizes (min/max/avg): 43,842/143,010/12

In [66]:
# Unpersist cached RDDs to free memory
cleaned_rdd.unpersist()
filtered_rdd.unpersist()
optimized_rdd.unpersist()
cleaned_rejected_rdd.unpersist()

print("Cached RDDs unpersisted.")

spark.stop()

Cached RDDs unpersisted.


## Next Steps

The Silver layer is complete. The data is now cleaned and ready for analytics and ML.

**Continue to:** `03_gold_serving.ipynb` for data serving using DataFrames, SQL, and MLlib.